# Исследование опроса клиентов телекоммуникационной компании 

### Описание и цель проекта
Перед компанией стояла задача определить текущий уровень потребительской лояльности, NPS, среди клиентов из России. 
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Оценки разделили на три группы:
* 9-10 баллов — promoters;
* 7-8 баллов — passives;
* 0-6 баллов — detractors.
### Описание данных 
* user_id - Идентификатор клиента
* lt_day - Количество дней «жизни» клиента
* is_new - Поле хранит информацию о том, является ли клиент новым
* age - Возраст
* gender_segment - Пол 
* os_name - Тип операционной системы
* cpe_type_name - Тип устройства
* country - Страна проживания
* city - Город проживания
* age_segment - Возрастной сегмент
* traffic_segment - Сегмент по объёму потребляемого трафика
* lifetime_segment - Сегмент по количеству дней «жизни»
* nps_score - Оценка клиента в NPS-опросе
* nps_group - Поле хранит информацию о том, к какой группе относится оценка клиента в опросе
### Декомпозиция проекта 
1. Подключиться к базе данных и сохранить данные в датафрейм
2. Собрать витрину данных на основании нескольких таблиц
3. Создать дашборд в Tableau, который будет хранить информацию о текущем уровне NPS среди клиентов и покажет, как этот уровень меняется в зависимости от пользовательских признаков
4. Сформулировать выводы 

In [2]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [3]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [4]:
query = """
SELECT u.user_id,
       u.lt_day,
       u.age,
       u.gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       a.title AS age_segment,
       t.title AS traffic_segment,
       lt.title AS lifetime_segment,
    CASE 
        WHEN u.lt_day < 365 THEN 'new'
        WHEN u.lt_day >= 365 THEN 'old'
    END AS is_new,
        u.nps_score,
    CASE
        WHEN nps_score >= 9 and nps_score <= 10 THEN 'promoters'
        WHEN nps_score >= 7 and nps_score <=8 THEN 'passives'
        WHEN nps_score >= 0 and nps_score <= 6 THEN 'detractors'
    END AS nps_group,
    CASE
        WHEN gender_segment = 1.0 and gender_segment NOT NULL THEN 'Женщина'
        WHEN gender_segment = 0.0 and gender_segment NOT NULL THEN 'Мужчина'
    END AS gender_segment
FROM user AS u
LEFT JOIN location AS l ON u.location_id=l.location_id
LEFT JOIN age_segment AS a ON u.age_gr_id=a.age_gr_id
LEFT JOIN traffic_segment AS t ON u.tr_gr_id=t.tr_gr_id
LEFT JOIN lifetime_segment AS lt ON u.lt_gr_id=lt.lt_gr_id
;

"""

In [5]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,is_new,nps_score,nps_group,gender_segment
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,old,10,promoters,Женщина
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,old,10,promoters,Мужчина
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,old,10,promoters,Мужчина
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,old,10,promoters,Женщина
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,old,10,promoters,Мужчина
5,A005O0,5501,42.0,1.0,ANDROID,SMARTPHONE,Россия,Омск,04 35-44,05 5-10,08 36+,old,6,detractors,Женщина
6,A0061R,1236,45.0,0.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,06 10-15,08 36+,old,10,promoters,Мужчина
7,A009KS,313,35.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,04 35-44,13 45-50,05 7-12,new,10,promoters,Мужчина
8,A00AES,3238,36.0,1.0,ANDROID,SMARTPHONE,Россия,СанктПетербург,04 35-44,04 1-5,08 36+,old,10,promoters,Женщина
9,A00F70,4479,54.0,1.0,ANDROID,SMARTPHONE,Россия,Волгоград,05 45-54,07 15-20,08 36+,old,9,promoters,Женщина


In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   age               501939 non-null  float64
 3   gender_segment    501192 non-null  float64
 4   os_name           502493 non-null  object 
 5   cpe_type_name     502493 non-null  object 
 6   country           502493 non-null  object 
 7   city              502493 non-null  object 
 8   age_segment       502493 non-null  object 
 9   traffic_segment   502493 non-null  object 
 10  lifetime_segment  502493 non-null  object 
 11  is_new            502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
 14  gender_segment    501192 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 57.5+ MB


In [8]:
df['gender_segment'] = df['gender_segment'].astype(str)

In [9]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

Ссылка на дашборд: https://public.tableau.com/views/Book3_16914392572000/Story1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link